<a href="https://colab.research.google.com/github/engineerJPark/FCN_Implementation/blob/main/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이미지 크기 어떻게 될지 살펴볼 것


https://github.com/wkentaro/pytorch-fcn/blob/ad22a47671140b2abc6712a0e004193fa382bc48/torchfcn/models/fcn16s.py#L78

dataset 참고할 것

https://pytorch.org/vision/stable/generated/torchvision.datasets.VOCSegmentation.html#torchvision.datasets.VOCSegmentation

COCO dataset

https://ndb796.tistory.com/667

Label Accuracy Code
https://github.com/wkentaro/pytorch-fcn/blob/main/torchfcn/utils.py

Loss and Train Code
https://github.com/wkentaro/pytorch-fcn/blob/main/torchfcn/trainer.py


In [ ]:
# test zone
# inputs = torch.randn((3,3,16,16))
# targets = torch.randint(3,(3,16,16), dtype=torch.float)
# y = F.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)
# y

x = torch.tensor([1, 2, 3])
# x.repeat(4, 2)
x.repeat(4, 2, 1)

tensor([[[1, 2, 3],
         [1, 2, 3]],

        [[1, 2, 3],
         [1, 2, 3]],

        [[1, 2, 3],
         [1, 2, 3]],

        [[1, 2, 3],
         [1, 2, 3]]])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights

from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset

# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

resnet_pretrained = resnet18(weights=ResNet18_Weights.DEFAULT)
print(resnet_pretrained)

In [ ]:
# # dataloader on COCO dataset

# # use name as train_set, val_set, test_set

# # need to be fixed
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#             ])

# cifar10_train = dset.CIFAR10('./datasets', train=True, download=True,
#                              transform=transform)
# loader_train = DataLoader(cifar10_train, batch_size=64, 
#                           sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

# cifar10_val = dset.CIFAR10('./datasets', train=True, download=True,
#                            transform=transform)
# loader_val = DataLoader(cifar10_val, batch_size=64, 
#                         sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

# cifar10_test = dset.CIFAR10('./datasets', train=False, download=True, 
#                             transform=transform)
# loader_test = DataLoader(cifar10_test, batch_size=64)

# Nin, Cin, Hin, Win, class_n = voc_train.shape

In [ ]:
# Bilinear weights deconvolution Algorithm 알아보기
def bilinear_kernel_init(Cin, Cout, kernel_size):
  factor = (kernel_size + 1) // 2
  if kernel_size % 2 == 1:
    center = (kernel_size + 1) // 2 - 1
  else:
    center = (kernel_size + 1) // 2 - 0.5

  og = (torch.arange(kernel_size).reshape(-1,1), torch.arange(kernel_size).reshape(1,-1))
  filter = (1 - torch.abs(og[0] - center) / factor) * (1 - torch.abs(og[1] - center) / factor)

  weight = torch.zeros((Cin, Cout, kernel_size, kernel_size))
  weight[range(Cin), range(Cout), :, :] = filter
  return weight


In [ ]:
class FCN18(nn.Module):
  def __init__(self, class_n):
    super().__init__()
    self.downsample1 = nn.Sequential( # 1/2 of the input
        resnet_pretrained.conv1,
        resnet_pretrained.bn1,
        resnet_pretrained.relu,
        resnet_pretrained.maxpool
    )
    self.downsample2 = resnet_pretrained.layer1 # 1/4 of the input
    self.downsample3 = resnet_pretrained.layer2 # 1/8 of the input, 여기서 1x1 거친것 하나 추출
    self.downsample4 = resnet_pretrained.layer3 # 1/16 of the input, 여기서 1x1 거친것 하나 추출
    self.downsample5 = resnet_pretrained.layer4 # 1/32 of the input, 여기서 1x1 거친것 하나 추출

    self.fully_conv_pool3 = nn.Conv2d(128, class_n, kernel_size=1) # get class score for each pixel
    nn.init.xavier_normal_(self.fully_conv_pool3.weight)
    # nn.init.xavier_normal_(self.fully_conv_pool3.bias)

    self.fully_conv_pool4 = nn.Conv2d(256, class_n, kernel_size=1) # get class score for each pixel
    nn.init.xavier_normal_(self.fully_conv_pool4.weight)
    # nn.init.xavier_normal_(self.fully_conv_pool4.bias)

    self.fully_conv_pool5 = nn.Conv2d(512, class_n, kernel_size=1) # get class score for each pixel
    nn.init.xavier_normal_(self.fully_conv_pool5.weight)
    # nn.init.xavier_normal_(self.fully_conv_pool5.bias)
    
    # 보통 stride s, padding s/2, kernelsize 2s -> s배 만큼 이미지가 커진다.
    # 이부분 튜닝해야한다.
    self.upsample_make_16s = nn.ConvTranspose2d(class_n, class_n, kernel_size=4, padding=1, stride=2, bias=False) # to 1/16
    self.upsample_make_16s.weight.data.copy_(bilinear_kernel_init(class_n, class_n, 4))

    self.upsample_make_8s = nn.ConvTranspose2d(class_n, class_n, kernel_size=4, padding=1, stride=2, bias=False) # to 1/8
    self.upsample_make_8s.weight.data.copy_(bilinear_kernel_init(class_n, class_n, 4))
    
    self.upsample_to_score = nn.ConvTranspose2d(class_n, class_n, kernel_size=16, padding=4, stride=8) # to 1
    self.upsample_to_score.weight.data.copy_(bilinear_kernel_init(class_n, class_n, 16))
    for param in self.upsample_to_score.parameters():
      param.requires_grad = False


  def forward(self, x):
    x = self.downsample1(x)
    x = self.downsample2(x)
    x = self.downsample3(x)
    pool3_score = self.fully_conv_pool3(x)
    x = self.downsample4(x)
    pool4_score = self.fully_conv_pool4(x)
    x = self.downsample5(x)
    pool5_score = self.fully_conv_pool5(x)

    pool4_2x_conv7 = pool4_score + self.upsample_make_16s(pool5_score)
    pool3_2x_pool4_4x_conv7 = pool3_score + self.upsample_make_8s(pool4_2x_conv7)

    out = self.upsample_to_score(pool3_2x_pool4_4x_conv7)

    return out

# model = FCN18(3)
# x = torch.rand([10, 3,1024,1024])
# print(model(x).shape)

In [ ]:
epochs = 10
lr = 10e-4
weight_decay = 2e-4
momentum = 0.9
batch_size = 20

model = FCN18(3)
optimizer = optim.SGD(model.paramters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def loss_func(self, inputs, targets):
  '''
  cross entropy to each feature vector
  inputs shape : (N, C, H, W)
  targets shape : (N, C, H, W)
  만약 tragets가 (N, H, W)이라면 : target.view(n, h, w, 1).repeat(1, 1, 1, c)
  '''
  feature_vector = F.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1) # output a feature vector : shape is (channel, )
  return feature_vector.mean()

def train():
  for epoch in range(epochs):
    for train_img, train_gt_img in train_set:
      score_img = model(train_img)
      loss = loss_func(score_img, train_gt_img)

      optimizer.zero_grad()
      optimizer.step()
      loss.backward()
    print("epoch %d, loss : %f "%(epoch + 1, loss))
  print("Training End")

In [ ]:
def validate():
  '''
  implement mean IU first 
  '''
  pass


def validate(self):
    training = self.model.training
    self.model.eval()

    n_class = len(self.val_loader.dataset.class_names)

    val_loss = 0
    visualizations = []
    label_trues, label_preds = [], []
    for batch_idx, (data, target) in tqdm.tqdm(
            enumerate(self.val_loader), total=len(self.val_loader),
            desc='Valid iteration=%d' % self.iteration, ncols=80,
            leave=False):
        if self.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        with torch.no_grad():
            score = self.model(data)

        loss = cross_entropy2d(score, target,
                                size_average=self.size_average)
        loss_data = loss.data.item()
        if np.isnan(loss_data):
            raise ValueError('loss is nan while validating')
        val_loss += loss_data / len(data)

        imgs = data.data.cpu()
        lbl_pred = score.data.max(1)[1].cpu().numpy()[:, :, :]
        lbl_true = target.data.cpu()
        for img, lt, lp in zip(imgs, lbl_true, lbl_pred):
            img, lt = self.val_loader.dataset.untransform(img, lt)
            label_trues.append(lt)
            label_preds.append(lp)
            if len(visualizations) < 9:
                viz = fcn.utils.visualize_segmentation(
                    lbl_pred=lp, lbl_true=lt, img=img, n_class=n_class)
                visualizations.append(viz)
    metrics = torchfcn.utils.label_accuracy_score(
        label_trues, label_preds, n_class)

    out = osp.join(self.out, 'visualization_viz')
    if not osp.exists(out):
        os.makedirs(out)
    out_file = osp.join(out, 'iter%012d.jpg' % self.iteration)
    skimage.io.imsave(out_file, fcn.utils.get_tile_image(visualizations))

    val_loss /= len(self.val_loader)

    with open(osp.join(self.out, 'log.csv'), 'a') as f:
        elapsed_time = (
            datetime.datetime.now(pytz.timezone('Asia/Tokyo')) -
            self.timestamp_start).total_seconds()
        log = [self.epoch, self.iteration] + [''] * 5 + \
              [val_loss] + list(metrics) + [elapsed_time]
        log = map(str, log)
        f.write(','.join(log) + '\n')

    mean_iu = metrics[2]
    is_best = mean_iu > self.best_mean_iu
    if is_best:
        self.best_mean_iu = mean_iu
    torch.save({
        'epoch': self.epoch,
        'iteration': self.iteration,
        'arch': self.model.__class__.__name__,
        'optim_state_dict': self.optim.state_dict(),
        'model_state_dict': self.model.state_dict(),
        'best_mean_iu': self.best_mean_iu,
    }, osp.join(self.out, 'checkpoint.pth.tar'))
    if is_best:
        shutil.copy(osp.join(self.out, 'checkpoint.pth.tar'),
                    osp.join(self.out, 'model_best.pth.tar'))

    if training:
        self.model.train()